# Map view with Dash

## Reference: [Brendon Hall](https://github.com/brendonhall/FORCE-2020-Lithology/blob/master/notebooks/03-Map-Dash.ipynb)

In [ ]:
import os.path
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
pd.options.display.max_rows = 8
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
data_path = os.environ.get("DATA_PATH")

In [ ]:
locations_df = pd.read_csv(data_path+'/force_2020_meta.csv')

In [ ]:
train_df = pd.read_csv(data_path+'/train.csv', sep=';')

test_df = pd.read_csv(data_path+'/test.csv', sep=';')

well_data_df = pd.concat([train_df, test_df])

In [ ]:
curves = ['FORMATION', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR',
          'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO',
          'MUDWEIGHT', 'RMIC', 'ROPA', 'RXO']

def well_has_curve(row, data_df, curve):
    well_name = row['WELL']
    return not data_df.loc[data_df['WELL']==well_name, curve].isnull().all()
    
for curve in curves:
    locations_df[curve] = locations_df.apply(lambda row: well_has_curve(row, well_data_df, curve), axis=1)

In [ ]:
# use Jupyter Dash to quickly build a prototype of the dash app right in this notebook
# for production or more complicated use we'll use the webserver version
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("FORCE 2020 Well Data Map"),
    html.Label([
        "Selected wells must contain:",
        dcc.Dropdown(
            id='curves-dropdown', clearable=False,
            options=[
                {'label': c, 'value': c}
                for c in curves
            ],
            value=['GR'],
            multi=True),
        dcc.Checklist(
            id='datasets-checkbox',
        options=[
            {'label': 'Training Data', 'value': 'Train'},
            {'label': 'Test Data', 'value': 'Test'}
            ],
            value=['Train', 'Test']),  
        dcc.Graph(id='map-plot'),
        
        # code-styled text area to display list of filtered well names
        html.Code(id='selected-wells-output', style={'whiteSpace': 'pre-line'})
    ]),   
])

In [ ]:
# Define callback to update graph
@app.callback(
    Output('map-plot', 'figure'),
    [Input("curves-dropdown", "value"),
     Input('datasets-checkbox', 'value')]
)
def update_figure(curves, datasets):

    curve_mask = locations_df[curves].values.all(axis=1)
    dff = locations_df[curve_mask]
    
    dff = dff[dff['Dataset'].isin(datasets)]
    
    num_wells = dff.shape[0]
    
    # if the selection is empty, just display the map centered at the 
    # same location
    if num_wells == 0:
        fig = px.scatter_mapbox(lat=[59.610511], lon= [2.772594],
                                center={'lat': 59.610511, 'lon': 2.772594},
                                zoom=5, height=600,
                                )

    else:
        fig = px.scatter_mapbox(dff, lat="lat", lon="lon",
                                center={'lat': 59.610511, 'lon': 2.772594},
                                color='Dataset',
                                zoom=5, height=600,
                                hover_data={'WELL': True,
                                            'lat': False,
                                            'lon': False,
                                            'Dataset': False,
                                            'Drilling Operator': True,
                                            'Purpose': True,
                                            'Completion Year': True,
                                            'Formation': True},
                                color_discrete_map={'Train': 'blue',
                                                    'Test': 'red'}
                                )

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":40,"t":50,"l":40,"b":0})

    return fig

In [ ]:
@app.callback(
    Output('selected-wells-output', 'children'),
    [Input("curves-dropdown", "value"),
     Input('datasets-checkbox', 'value'),
     Input('map-plot', 'selectedData')]
)
def update_output(curves, datasets, selected_items):
    
    # return the well names for all wells that match the criterion
    curve_mask = locations_df[curves].values.all(axis=1)
    dff = locations_df[curve_mask]
    dff = dff[dff['Dataset'].isin(datasets)]
    # now filter by wells that have been selected in the image.
    all_wells = dff['WELL'] # initialize to all of the wells
    lasso_wells = None
    
    # now iterate over selected items, and get the relevant point data
    # need to do some checks to make sure valid data selected
    if selected_items is not None:
        for selected_item in [selected_items]:
            if selected_item and selected_item['points']:
                # get a list of the selected well names
                lasso_wells = [p['customdata'][0] for p in selected_item['points']]
   
    select_mask = dff['WELL'].isin(lasso_wells if lasso_wells else all_wells)
    dff = dff[select_mask]
    
    num_wells = dff.shape[0]
    well_names = dff['WELL'].values

    output_text = "\nYou have selected {} wells.\n\n".format(num_wells)
    well_list_text = ", ".join("'{}'".format(well_name) for well_name in well_names)
    well_list_text = "selected_wells = ["+well_list_text + "]"
    return output_text + well_list_text

In [ ]:
app.run_server(mode='external')

## References
Bormann P., Aursand P., Dilib F., Dischington P., Manral S. (2020) 2020 FORCE Machine Learning Contest. https://github.com/bolgebrygg/Force-2020-Machine-Learning-competition